# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://docs.dask.org/en/latest/generated/dask.dataframe.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35065,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:34339,Total threads: 4
Dashboard: http://127.0.0.1:37541/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:40133,


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,983,Patricia,0.322972,0.600058
1,985,Hannah,-0.910913,-0.151170
2,1010,Wendy,-0.923381,-0.258776
3,1006,Charlie,-0.273582,-0.421710
4,1016,Bob,-0.297881,0.095210


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,1.000024e+03,-3.014975e-04,-7.060501e-04
std,3.160784e+01,5.773494e-01,5.772475e-01
min,8.400000e+02,-9.999996e-01,-9.999976e-01
25%,9.790000e+02,-4.954986e-01,-4.942786e-01
50%,1.000000e+03,5.131531e-03,5.806145e-03
75%,1.022000e+03,5.071223e-01,5.039019e-01
max,1.171000e+03,9.999996e-01,1.000000e+00


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
1005,Tim,-0.9837186200258344
1056,Frank,-0.2578760758261991
980,Wendy,0.671073065086307
1006,Ingrid,0.4857500037355269
986,Xavier,0.8785289661766846
1032,Alice,-0.5921816520037597
998,Quinn,0.4018080680457687
993,Ingrid,-0.0444781268910403
982,Victor,0.5402867220503078


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,983,Patricia,0.322972,0.600058
1,985,Hannah,-0.910913,-0.151170
2,1010,Wendy,-0.923381,-0.258776
3,1006,Charlie,-0.273582,-0.421710
4,1016,Bob,-0.297881,0.095210


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   []
  Missing: ['y']

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,983,Patricia,0.322972,0.600058
1,985,Hannah,-0.910913,-0.151170
2,1010,Wendy,-0.923381,-0.258776
3,1006,Charlie,-0.273582,-0.421710
4,1016,Bob,-0.297881,0.095210


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,1005,Tim,-0.983719,NaN
1,1056,Frank,-0.257876,NaN
2,980,Wendy,0.671073,NaN
3,1006,Ingrid,0.485750,NaN
4,986,Xavier,0.878529,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-ce525c5e-2c71-416d-ae2a-3168aac3d938')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,983,Patricia,0.322972,0.600058
1,985,Hannah,-0.910913,-0.151170
2,1010,Wendy,-0.923381,-0.258776
3,1006,Charlie,-0.273582,-0.421710
4,1016,Bob,-0.297881,0.095210
...,...,...,...,...
86395,966,Laura,0.226231,-0.961098
86396,989,Charlie,-0.052508,-0.555283
86397,962,Frank,0.601037,-0.194042
86398,989,Laura,0.576488,0.435888


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-27.csv',
 'data/2000-01-15.csv',
 'data/2000-01-10.csv',
 'data/2000-01-01.csv',
 'data/2000-01-13.csv',
 'data/2000-01-14.csv',
 'data/2000-01-21.csv',
 'data/2000-01-30.csv',
 'data/2000-01-29.csv',
 'data/2000-01-28.csv',
 'data/2000-01-20.csv',
 'data/2000-01-11.csv',
 'data/2000-01-07.csv',
 'data/2000-01-22.csv',
 'data/2000-01-08.csv',
 'data/2000-01-02.csv',
 'data/2000-01-04.csv',
 'data/2000-01-05.csv',
 'data/2000-01-17.csv',
 'data/2000-01-23.csv',
 'data/2000-01-26.csv',
 'data/2000-01-16.csv',
 'data/2000-01-24.csv',
 'data/2000-01-03.csv',
 'data/2000-01-06.csv',
 'data/2000-01-18.csv',
 'data/2000-01-12.csv',
 'data/2000-01-19.csv',
 'data/2000-01-25.csv',
 'data/2000-01-09.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-a91410c0-ff73-4d0d-abab-a535df52b5ec'),
 Delayed('read_data-5c3744a0-a439-434d-911b-59444a48c520'),
 Delayed('read_data-22c9beca-49a7-4307-b75c-45a28a559b46'),
 Delayed('read_data-5904ca64-bd67-4552-980b-5977905b9bde'),
 Delayed('read_data-8ba97c0a-0894-4a53-ba35-15406b021818'),
 Delayed('read_data-cc2f9dfe-5d2b-4045-a20c-2a51deff8639'),
 Delayed('read_data-17f6dd8a-d7f2-4924-97d7-74107ec9d572'),
 Delayed('read_data-6b994fe9-2263-439b-911b-031fa35ec164'),
 Delayed('read_data-ea260c53-143f-49c0-861d-89990ecc93e5'),
 Delayed('read_data-487e673b-4f22-4c73-9fc7-a89812be50de'),
 Delayed('read_data-44202dcd-b7af-4f0c-9a70-837ff2f44ac2'),
 Delayed('read_data-3b3250b1-8a9c-4201-bdf9-9f81db8b8884'),
 Delayed('read_data-54af63eb-80b1-4125-b249-67f3b175b5ae'),
 Delayed('read_data-73f33bd1-b08e-4dd6-87ab-544a5bfbd05e'),
 Delayed('read_data-686598cc-cbbf-44fb-a6d2-72fa70ec35ff'),
 Delayed('read_data-5e974513-f343-441e-a3f4-493446f86a09'),
 Delayed('read_data-efb79e36-32aa-40bc-a

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/generated/dask.dataframe.from_delayed.html). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,1002,Ursula,-0.360814,-0.967969
1,948,Tim,-0.506405,-0.262879
2,967,Jerry,0.809394,0.793698
3,1022,Patricia,0.646362,-0.175778
4,1065,Kevin,0.038873,0.317031


In [26]:
df.describe().compute()

,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,1.000025e+03,-3.309586e-04,-8.221903e-04
std,3.160755e+01,5.773756e-01,5.772494e-01
min,8.400000e+02,-9.999996e-01,-9.999976e-01
25%,9.790000e+02,-4.954986e-01,-4.942786e-01
50%,1.000000e+03,5.131531e-03,5.806145e-03
75%,1.022000e+03,5.071223e-01,5.039019e-01
max,1.171000e+03,9.999996e-01,9.999993e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.